In [ ]:
#!pip install ipython-sql
#!pip install ibm_db 
#!pip install ibm_db_sa
#!pip install beautifulsoup4
import ibm_db
import ibm_db_sa
import sqlalchemy
import pandas as pd
import datetime as dt   

from urllib.request import urlopen
from bs4 import BeautifulSoup

#%load_ext sqly
%reload_ext sql



In [ ]:
date_start = dt.date(2021, 1, 1)
date_end = dt.date(2021,1, 26)

date_pool = pd.date_range(date_start, date_end)

k = 0

for i in date_pool:
    path_result = "https://lovifogadas.hu/eredmenyek/?datum="+(str(i)[:10])+"&palya=1#"
    try:
        
        felallas_data = pd.read_html(path_result)[1]
    
        for j in range(0,felallas_data.index.max()):
            
            szam = str(felallas_data.iloc[j]['Szám'])
            lo_neve = felallas_data.iloc[j]['Ló neve']
            zsoke = felallas_data.iloc[j]['Hajtó / Zsoké']
            datum = str(i)
            szorzo = felallas_data.iloc[j,5]
            
            %sql  INSERT INTO felallas \
            (id,szam,lo,zsoke,datum,szorzo) \
            VALUES (:k,:szam,:lo_neve,:zsoke,:datum,:szorzo)
        
        
    except:
        print(path_result)
    k = k+1

In [ ]:



date_start = dt.date(2020, 4, 1)
date_end = dt.date(2021,2, 5)


date_pool = pd.date_range(date_start, date_end)

k = 284

for today in date_pool:
    pre_url = "https://lovifogadas.hu/eredmenyek/?datum="+(str(today)[:10])+"&palya=1#"
    pre_html = urlopen(pre_url)
    pre_soup = BeautifulSoup(pre_html,'lxml')
    series = []
    for option in pre_soup.find_all('option'):
        series.append(option['value'])
        
    for j in series:
        
            url = "https://lovifogadas.hu/eredmenyek/?datum="+(str(today)[:10])+"&palya="+str(j)+"#"
            html = urlopen(url)

            soup = BeautifulSoup(html,'lxml')

            result_temp = pd.DataFrame(columns =  ['id','datum','Pálya neve', 'Futam neve','Start', \
                                                   'Futamszám','Lejelentések','Lefutási idő',\
                                                   'elso','masodik','harmadik','negyedik','otodik'])

            
            for td in soup.find_all("td"): 
                result_temp['id'] = k
                result_temp['datum'] = today
                if td.get('data-th') == 'Pálya neve':
                    result_temp = result_temp.append({'Pálya neve':td.text},ignore_index = True)
                elif td.get('data-th') == 'Futam neve':
                    result_temp = result_temp.append({'Futam neve':td.text},ignore_index = True)
                elif td.get('data-th') == 'Start':
                    result_temp = result_temp.append({'Start':str(td.text)[:5]},ignore_index = True)
                elif td.get('data-th') == 'Futamszám':
                    result_temp = result_temp.append({'Futamszám':td.text},ignore_index = True)
                elif td.get('data-th') == 'Lejelentések':
                    result_temp = result_temp.append({'Lejelentések':td.text},ignore_index = True)
                elif td.get('data-th') == 'Befutási sorrend':
                    sorrend = td
                    s_temp = []
                    for s in sorrend:
                        if (s.find_all(title='Holtverseny') ):
                            s_temp.append(s.text)
                        else:
                            s_temp.append(s.text)
                            
                    try:
                        result_temp = result_temp.append(      \
                        {'elso':s_temp[0], 'masodik':s_temp[1],"harmadik":s_temp[2],"negyedik":s_temp[3],"otodik":s_temp[4]}   \
                        ,ignore_index = True)
            
                    except IndexError:
                        try:
                            
                            result_temp = result_temp.append(      \
                            {'elso':s_temp[0], 'masodik':s_temp[1],"harmadik":s_temp[2],"negyedik":s_temp[3],"otodik":"0"}   \
                            ,ignore_index = True)
                            
                        except IndexError:
                            
                            try:
                                
                                result_temp = result_temp.append(      \
                                {'elso':s_temp[0], 'masodik':s_temp[1],"harmadik":s_temp[2],"negyedik":"0","otodik":"0"}   \
                                ,ignore_index = True)
                                
                            except IndexError:
                                
                                try: 
                                    
                                    result_temp = result_temp.append(      \
                                    {'elso':s_temp[0], 'masodik':s_temp[1],"harmadik":"0","negyedik":"0","otodik":"0"}   \
                                    ,ignore_index = True)
                                    
                                    
                                except IndexError:
                                    
                                    try:
                                        
                                        result_temp = result_temp.append(      \
                                        {'elso':s_temp[0], 'masodik':"0","harmadik":"0","negyedik":"0","otodik":"0"}   \
                                        ,ignore_index = True)
                                        
                                    except IndexError:
                                        
                                        result_temp = result_temp.append(      \
                                        {'elso':"0", 'masodik':"0","harmadik":"0","negyedik":"0","otodik":"0"}   \
                                        ,ignore_index = True)               
                        
 
                elif td.get('data-th') == 'Lefutási idő':
                    result_temp = result_temp.append({'Lefutási idő':td.text},ignore_index = True)
    
            
            result_temp = result_temp.replace(to_replace='', value='0', regex=True) #https://lovifogadas.hu/eredmenyek/?datum=2019-09-28&palya=2#
        
            palya_neve = result_temp[result_temp['Pálya neve'].notna() == True]
            futam_neve = result_temp[result_temp['Futam neve'].notna() == True]
            start = result_temp[result_temp['Start'].notna() == True]
            futamszam = result_temp[result_temp['Futamszám'].notna() == True]
            lejelentesek = result_temp[result_temp['Lejelentések'].notna() == True]
            lefutasi_ido = result_temp[result_temp['Lefutási idő'].notna() == True]
            elso = result_temp[result_temp['elso'].notna() == True]
            masodik = result_temp[result_temp['masodik'].notna() == True]
            harmadik = result_temp[result_temp['harmadik'].notna() == True]
            negyedik = result_temp[result_temp['negyedik'].notna() == True]
            otodik = result_temp[result_temp['otodik'].notna() == True]
            
            
            
            palya_neve.reset_index(inplace = True,drop=True)
           

            for i in range(0,palya_neve.index.max()+1):
    
                ix = int(palya_neve['id'].iloc[i])
                d = str(palya_neve['datum'].iloc[i])
                p = palya_neve['Pálya neve'].iloc[i]
                f = futam_neve['Futam neve'].iloc[i]
                s = start['Start'].iloc[i]
                fsz = futamszam['Futamszám'].iloc[i]
                l = lejelentesek['Lejelentések'].iloc[i]
                it = lefutasi_ido['Lefutási idő'].iloc[i]
                e = elso['elso'].iloc[i]
                m = masodik['masodik'].iloc[i]
                h = harmadik['harmadik'].iloc[i]
                n = negyedik['negyedik'].iloc[i]
                o = otodik['otodik'].iloc[i]
                
                
    
                %sql  INSERT INTO eredmenyek \
                    (id,datum,palya,futam,start,futamszam,lejelentesek,idotartam,elso,masodik,harmadik,negyedik,otodik) \
                    VALUES(:ix,:d,:p,:f,:s,:fsz,:l,:it,:e,:m,:h,:n,:o)
                
        
    
    k = k+1


In [ ]:
%sql select * from eredmenyek